In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, Row

In [2]:
spark = SparkSession.builder.appName('dataProcessing2').config('spark.executor.memory', '6g').getOrCreate()

23/11/26 19:01:27 WARN Utils: Your hostname, Bhavins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.123 instead (on interface en0)
23/11/26 19:01:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/26 19:01:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
business_df = spark.read.json('Data/yelp_academic_dataset_business.json')
business_df = business_df.select('business_id', 'name', 'postal_code', 'latitude', 'longitude', 'stars')
business_df = business_df.withColumnRenamed('stars', 'business_stars')
review_df = spark.read.json('Data/yelp_academic_dataset_review.json')
review_df = review_df.select('review_id', 'user_id', 'business_id', 'stars')
review_df = review_df.withColumnRenamed('stars', 'review_stars')
# user_df = spark.read.json('Data/yelp_academic_dataset_user.json')
# user_df = user_df.select('user_id', 'name', 'fans','friends')
# user_df = user_df.withColumnRenamed('name', 'user_name')
merged_df = review_df.join(business_df, 'business_id', 'inner')
# merged_df = merged_df.join(user_df, 'user_id', 'inner')
merged_df.head(5)
#merged_df.write.csv('Data/merged_data.csv', header=True)

[Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', review_id='KU_O5udG6zpxOg-VcAEodg', user_id='mh_-eMZ6K5RLWhZyISBhwA', review_stars=3.0, name='Turning Point of North Wales', postal_code='19454', latitude=40.2101961875, longitude=-75.2236385919, business_stars=3.0),
 Row(business_id='7ATYjTIgM3jUlt4UM3IypQ', review_id='BiTunyQ73aT9WBnpR9DZGw', user_id='OyoGAe7OKpv6SyGZT5g77Q', review_stars=5.0, name='Body Cycle Spinning Studio', postal_code='19119', latitude=39.9521029, longitude=-75.1727526, business_stars=5.0),
 Row(business_id='YjUWPpI6HXG530lwP-fb2A', review_id='saUsX_uimxRlCVr67Z4Jig', user_id='8g_iMtfSiwikVnbP2etR0A', review_stars=3.0, name='Kettle Restaurant', postal_code='85713', latitude=32.2072326, longitude=-110.9808639, business_stars=3.5),
 Row(business_id='kxX2SOes4o-D3ZQBkiMRfA', review_id='AqPFMleE6RsU23_auESxiA', user_id='_7bHUi9Uuf5__HHc_Q8guQ', review_stars=5.0, name='Zaika', postal_code='19114', latitude=40.0798480557, longitude=-75.025079772, business_stars=4.0),
 Row(bus

In [9]:
# merged_df = spark.read.csv('Data/merged_data.csv', header=True)
review_df = spark.read.json('Data/yelp_academic_dataset_review.json')
print((review_df.count(), len(review_df.columns)))


(6990280, 9)


23/11/26 22:49:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 241138 ms exceeds timeout 120000 ms
23/11/26 22:49:32 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/26 22:49:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# user_item_df = merged_df.select('user_id', 'business_id', 'review_stars')
# spark.conf.set('spark.sql.pivotMaxValues', merged_df.select('business_id').distinct().count())
# # Sample a fraction of the data
# # For example, 0.1 to sample 10% of the data
# sampled_user_item_df = user_item_df.sample(withReplacement=False, fraction=0.1)
# sampled_user_item_df = sampled_user_item_df.groupBy('user_id').pivot('business_id').sum('review_stars')
# sampled_user_item_df = sampled_user_item_df.fillna(0)
# sampled_user_item_df.head(5)

In [17]:
#filtered_df.printSchema()
# For example, 0.1 to sample 10% of the data
user_item_df = merged_df.select('user_id', 'business_id', 'review_stars')
sampled_user_item_df = user_item_df.sample(withReplacement=False, fraction=0.0001)
sampled_user_item_df = sampled_user_item_df.withColumn('review_stars', sampled_user_item_df['review_stars'].cast('float'))
sampled_user_item_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- review_stars: float (nullable = true)



In [18]:
# Split the list into chunks (define your own chunk size)

from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType

from pyspark.sql.functions import col # for indicating a column using a string in the line below
chunk_size = 100  # Adjust this based on your memory constraints
distinct_business_ids = merged_df.select("business_id").distinct().collect() 
chunks = [distinct_business_ids[i:i + chunk_size] for i in range(0, len(distinct_business_ids), chunk_size)]

# Initialize an empty DataFrame to hold the final result
final_df = None
# Iterate over each chunk and perform the pivot operation
for chunk in chunks:
    # Create a DataFrame from the chunk
    chunk_df = spark.createDataFrame([b_id for b_id in chunk], ["business_id"])

    # Join with the original DataFrame
    filtered_df = chunk_df.join(sampled_user_item_df, chunk_df.business_id == sampled_user_item_df.business_id, "left").drop(sampled_user_item_df.business_id)
    # Pivot the filtered DataFrame
    pivoted_df = filtered_df.groupBy("user_id").pivot("business_id").sum("review_stars")

    # Join with the final DataFrame
    if final_df is None:
        final_df = pivoted_df
    else:
        final_df = final_df.join(pivoted_df, "user_id", "outer")


23/11/26 14:40:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 163942 ms exceeds timeout 120000 ms
23/11/26 14:40:01 WARN SparkContext: Killing executors is not supported by current scheduler.


Py4JJavaError: An error occurred while calling o4775.join.
: java.lang.OutOfMemoryError: GC overhead limit exceeded


In [ ]:
final_df.head(5)